# Advanced SQL III: Correlated Sub-Queries and Window Functions
_**Author**: Boom Devahastin Na Ayudhya_
***

Alright, it's the final stretch! This is the last of the three-part workshop on Advanced SQL techniques. This is going to be the toughest out of all topics we've covered, but we'll get through it!

## Warm-Up

**Warm-Up Exercise:**
Write a query that shows the department and the number of people in each department.

_Answer:_
```MySQL
SELECT department, COUNT(employee_id)
FROM employees
GROUP BY department
```

**But what about this:** Write a query that includes each employee's first_name, their department and the number of people in their department.

Some of you might be thinking you can just add `first_name` as an additional column and throw that into the `GROUP BY` like:
```MySQL
SELECT  first_name,
        department,
        COUNT(department)
FROM employees
GROUP BY department, first_name
```
But why does this **_NOT_ work**?

Here's the **right version** which makes use of sub-queries:
```MySQL
SELECT  first_name,
        department,
        (SELECT COUNT(department)
         FROM employees e1
         WHERE e1.department = e2.department)
FROM employees e2
GROUP BY department, first_name
```
This is what we call a **correlated sub-query**, which is a little complicated but useful. However, correlated sub-queries can be computationally inefficient because here it has to run for every single row! So...what's better?

## Window Functions!

Just to be clear a window function is **NOT** a function called `WINDOW()` - it is a family of functions that _**operate on a group of rows (window) that are somehow related to the current row**_.

This necessarily means `GROUP BY` is being used **_behind the scenes_** but we do not necessarily need to explicitly call `GROUP BY` anymore when we use window functions!

**IMPORTANT WARNING:** Depending on the SQL dialect used at your company, you may not have access to Window functions. PostgreSQL (which we're using here) supports it; however, MySQL does not.

### `OVER()`: Basic Uses
The most common Window Function is `OVER()` which allows us to specify the "window" (or group of rows) that is the focus of our analysis.

_Example:_
```MySQL
SELECT  first_name,
		department,
		COUNT(*) OVER(PARTITION BY department)
FROM employees
```

Let's check that the output of the correlated sub-query method we used earlier is equivalent to this:

```MySQL
(SELECT  first_name,
        department,
        (SELECT COUNT(department)
         FROM employees e1
         WHERE e1.department = e2.department)
FROM employees e2
GROUP BY department, first_name)

EXCEPT

(SELECT  first_name,
		department,
		COUNT(*) OVER(PARTITION BY department)
FROM employees)
```

**Exercise 1:** Write a query that includes each employee's first_name, their department and the total salaries earned of people in their department using

**(a) The Correlated Sub-Query Method**

_Answer:_
```MySQL
SELECT  first_name,
        department,
        (SELECT SUM(salary)
         FROM employees e1
         WHERE e1.department = e2.department)
FROM employees e2
GROUP BY department, first_name
```

**(b) The Window Function Method**

_Answer:_
```MySQL
SELECT  first_name,
        department,
        SUM(salary) OVER(PARTITION BY department)
FROM employees
```

**Exercise 2:** Write a query that includes:
- each employee's first_name
- their department
- their department size
- their region_id
- the total salaries earned of people in their region

using as few lines of code as possible.

_Answer:_
```MySQL
SELECT first_name,
	   department,
	   COUNT(*) OVER(PARTITION BY department) AS "dept_size",
       region_id,
	   SUM(salary) OVER(PARTITION BY region_id) AS "region_total_salary"
FROM employees
```

### `OVER()`: Cumulative Sums

Beyond the basic uses of `OVER()` as a more flexible `GROUP BY`, we can also use it to help us do cumulative calculations. 

_Example:_ Write a query that returns the first name of all employees, the hire_date, the employee's salary, and **the total salaries earned by employees on each date** (call it "cumulative_salary").

```MySQL
SELECT  first_name,
        hire_date,
        salary,
        SUM(salary) OVER(ORDER BY hire_date -- this is the index we are going by
                         RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_salary -- the input range
FROM employees
```

The `UNBOUNDED PRECEDING` here means from the start of the ordered series (earliest hire_date in this case).

**Exercise 3:** Write a query that returns the first name, hire date, salary, and cumulative salaries paid at each date for each department since the company was founded.

_Answer:_
```MySQL
SELECT  department, hire_date, first_name, salary,
        SUM(salary) OVER(PARTITION BY department -- equivalent of GROUP BY
                         ORDER BY hire_date -- index
                         RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_dept_salary -- input range
FROM employees
```

### `OVER()`: Rolling Averages

This is very similar to cumulative sums except we just need to change `SUM()` $\rightarrow$ `AVG()` and change `UNBOUNDED PRECEEDING` to a specified window.

It makes more sense to work with financial data to do this so let's switch gears to the `DailyQuote` table which you can access if you create a new database using commands from `DailyQuote_essentialsql.txt` _(Source: https://www.essentialsql.com/sql-puzzle-calculate-moving-averages/)_

*Example:* Calculate the 3-day rolling average closing price of the stock at each date.
```MySQL
SELECT  MarketDate,
        ClosingPrice,
        AVG(ClosingPrice) OVER (ORDER BY MarketDate ASC
                                ROWS BETWEEN '2' PRECEDING AND CURRENT ROW) AS "3D_roll_avg"
FROM DailyQuote
```

_**Aside: ...but technically the right way to do things**_

Notice that the rolling averages for the first 3 dates don't make sense since it's not a 5-day average but based on just the existing value up until that date.

```MySQL
SELECT MarketDate,
	   RowNumber,
	   ClosingPrice,
	   CASE WHEN RowNumber > 4 THEN "5D_roll_avg"
			ELSE NULL
			END AS "5D_roll_avg"
FROM (SELECT MarketDate,
			ClosingPrice,
	  ROW_NUMBER() OVER(ORDER BY MarketDate ASC) AS RowNumber,
	  AVG(ClosingPrice) OVER (ORDER BY MarketDate ASC
							  ROWS BETWEEN '4' PRECEDING AND CURRENT ROW) AS "5D_roll_avg"
	  FROM DailyQuote) AS subquery
```

**Exercise 4:** Calculate the 10-day and 30-day rolling average closing price of the stock at each date.
_(You can try this the quick way, or as a bonus try doing this the right way)_ 

_Answer:_
```MySQL
SELECT MarketDate,
	   RowNumber,
	   ClosingPrice,
	   CASE WHEN RowNumber > 9 THEN "10D_roll_avg"
			ELSE NULL
			END AS "10D_roll_avg",
	   CASE WHEN RowNumber > 29 THEN "30D_roll_avg"
			ELSE NULL
			END AS "30D_roll_avg",
FROM (SELECT MarketDate,
			 ClosingPrice,
	  ROW_NUMBER() OVER(ORDER BY MarketDate ASC) AS RowNumber,
	  AVG(ClosingPrice) OVER (ORDER BY MarketDate ASC
							  ROWS BETWEEN '9' PRECEDING AND CURRENT ROW) AS "10D_roll_avg",
	  AVG(ClosingPrice) OVER (ORDER BY MarketDate ASC
							  ROWS BETWEEN '29' PRECEDING AND CURRENT ROW) AS "30D_roll_avg"
	  FROM DailyQuote) AS subquery
```